In [4]:
from IPython.core.display import display, HTML
display(HTML('<style>.container {width:80% !important;}</style>'))

In [32]:
#get a large csv table of all the large HST images
data_dict = {"pubID":[], "obj_name":[] } 


import pandas as pd

from daomop.storage import tap_query
import os
import time 
import subprocess
from astropy import wcs
from astropy.io import fits
from astropy.nddata.utils import Cutout2D
import numpy as np
import random
from sklearn.preprocessing import minmax_scale


#OUT_PATH ='/run/media/toyonagar/Lexar/real_lenses/normalized/real_out'

CSV_PATH="test_csv"



def get_filenames(data_dict):
    query = '''SELECT caom2.Plane.publisherID, caom2.Plane.planeURI 
    FROM caom2.observation 
    JOIN caom2.plane ON caom2.observation.obsID = caom2.plane.obsID 
    WHERE caom2.observation.collection = 'HST' 
    AND caom2.observation.instrument_name = 'ACS/WFC'
    AND caom2.Plane.energy_bandpassName = 'F814W'
    '''
    #AND caom2.Plane.energy_bandpassName = 'F775W' 

    out_query = tap_query(query)

    '''store the pubID and filenames'''
    
    for entry in range(len(out_query["publisherID"])):#len(out_query["publisherID"])):
        if ("0-" in out_query["publisherID"][entry]) and ("PRODUCT" in out_query["publisherID"][entry]):
            data_dict["pubID"].append(out_query["publisherID"][entry])
            start = out_query["publisherID"][entry].find('HST?') + 4 #add 4 to get to the and of the HST?
            end = out_query["publisherID"][entry].find('/j', start)
            data_dict["obj_name"].append(out_query["publisherID"][entry][start:end])
            
    df= pd.DataFrame(data_dict)
    df.to_csv(CSV_PATH, sep=',')  #saving filenames to CSV      
    return data_dict


  
'''tap query for the potential results'''
start=time.time()
data_dict = get_filenames(data_dict)
    
print(time.time()-start)


        

2.46382904053


In [ ]:
#download the images form th master csv table

batch_size = 3
start_point = 3
DWNLD_PATH = "/run/media/toyonagar/Lexar/hst_detection/hst_det_"
CSV_PATH="test_csv"


im_info = pd.read_csv(CSV_PATH)

def dwnld(entry): 
    subprocess.check_output(["wget", "-O",DWNLD_PATH +str(entry)+ ".fits" ,
                             "http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/MAST/HST/product/"+
                             im_info['obj_name'][entry]+"_drz.fits"]) 
    
'''download image cutouts based on the file names'''
fails=0
for i in range(batch_size):
    print("dwnlding image " + str(i+start_point))
    try: 
        dwnld(i+start_point)
    except:
        fails+=1
        print("Failed")
print("failed: "+str(fails))
            



dwnlding image 3
dwnlding image 4


In [28]:
im_info = pd.read_csv("test_csv")
im_info['obj_name'][0:4]

0    jdn012010
1    jc9q01020
2    jca9ga020
3    jb6u05010
Name: obj_name, dtype: object

In [ ]:
http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/MAST/HST/product/jcl403f8q_flt.fits

In [42]:
pie = "pie is good"
pie.split()

['pie', 'is', 'good']

In [9]:
import subprocess

subprocess.check_output(["wget", "-O","test.fits" ,
                         "http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/caom2ops/cutout?uri=mast:HST/product/"+
                         "jcc212010_drz.fits&cutout=Circle+ICRS+" + str(14.670393222633267) + "+" +
                         str(-74.05824472139092) + "+0.001"])


''